In [14]:
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output, State
from CRUD_module import CRUD
import pandas as pd
from jupyter_dash import JupyterDash
import dash_leaflet as dl
import plotly.express as px

# Database setup
username = "aacuser"
password = "aacPass"
db = CRUD(username, password)
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Dash app layout
app = JupyterDash('Grazioso Salvare')

app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare'))),
    html.Center(html.Img(src="assets/Grazioso Salvare Logo.png", height=100)),
    html.Hr(),
    # Rescue Type Filter
    html.Div([
        html.Label("Select Rescue Type"),
        dcc.RadioItems(
            id='rescue-type-radio',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'all'}
            ],
            value='all'
        ),
    ]),
    # Dog Breed Dropdown
    html.Div([
        html.Label("Select Preferred Dog Breed"),
        dcc.Dropdown(
            id='dog-breed-dropdown',
            options=[
                {'label': label, 'value': label} for label in df[df['animal_type'] == 'Dog']['breed'].unique()
            ],
            value='all'
        ),
    ]),

    html.Br(),
    html.Button('Update', id='update-button'),
    html.Div(id='test-output'),

    # DataTable with highlighted row
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        page_size=10,
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        }
    ),
    # Map Visualization
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),
    # Pie Chart
    dcc.Graph(
        id='breed-pie-chart',
        figure={}
    )
])
# Criteria for filtering based on rescue type
RESCUE_CRITERIA = {
    'Water': {
        'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'gender': 'Intact Female',
        'age_range': (26, 156)
    },
    'Mountain': {
        'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'gender': 'Intact Male',
        'age_range': (26, 156)
    },
    'Disaster': {
        'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
        'gender': 'Intact Male',
        'age_range': (20, 300)
    }
}

@app.callback(
    [Output('table', 'data'),
     Output('breed-pie-chart', 'figure'),
     Output('test-output', 'children')],
    [Input('update-button', 'n_clicks')],
    [State('rescue-type-radio', 'value'),
     State('dog-breed-dropdown', 'value')]
)
def update_output(n_clicks, rescue_type, dog_breed):
    filtered_df = df

    # Filter by rescue type
    if rescue_type in RESCUE_CRITERIA:
        criteria = RESCUE_CRITERIA[rescue_type]
        filtered_df = df[df['breed'].isin(criteria['breeds']) & 
                         (df['age_upon_outcome_in_weeks'] >= criteria['age_range'][0]) & 
                         (df['age_upon_outcome_in_weeks'] <= criteria['age_range'][1]) & 
                         (df['sex_upon_outcome'] == criteria['gender'])]

    # Filter by dog breed
    if dog_breed != 'all':
        filtered_df = filtered_df[filtered_df['breed'] == dog_breed]

    # Create pie chart with combined small categories
    pie_data = filtered_df['breed'].value_counts(normalize=True).reset_index()
    pie_data.columns = ['breed', 'percent']
    threshold = 0.02
    mask = pie_data['percent'] > threshold
    tail = pie_data.loc[~mask]
    others = pd.DataFrame({'breed': ['Others'], 'percent': [tail['percent'].sum()]})
    pie_data = pd.concat([pie_data[mask], others], ignore_index=True)
    pie_chart_figure = px.pie(pie_data, names='breed', values='percent', title='Distribution of Breeds')

    return filtered_df.to_dict('records'), pie_chart_figure, f"Filtered by: {rescue_type} and {dog_breed}"

@app.callback(
    Output('table', 'style_data_conditional'),
    [Input('table', 'selected_rows')]
)
def highlight_selected_row(selected_rows):
    if not selected_rows:
        return []

    return [{
        'if': {'row_index': selected_rows[0]},
        'backgroundColor': 'yellow',
        'color': 'black'
    }]
@app.callback(
    Output('map-id', 'children'),
    [Input('table', 'selected_rows'),
     Input('table', 'data')]
)
def update_geolocation(selected_rows, table_data):
    dff = pd.DataFrame.from_dict(table_data)
    if not selected_rows:
        row = 0
    else:
        row = selected_rows[0]

    animal_name = dff.iloc[row, 9]
    animal_breed = dff.iloc[row, 4]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                       dl.Tooltip(f"Breed: {animal_breed}, Name: {animal_name}"),
                       dl.Popup([
                           html.H1("Animal Name"),
                           html.P(animal_name)
                       ])
                   ])
               ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True)


Dash is running on http://127.0.0.1:12229/

Dash app running on http://127.0.0.1:12229/
